# Cloud Workshop Azure Databricks
## 14. MLflow Serving Models with Azure ML service

<img src="https://raw.githubusercontent.com/retkowsky/images/master/AzureDatabricksLogo.jpg"><br>
V1.3 12/06/2019

## 0. Setup

1. Ensure you are using or create a cluster specifying 
  * **Databricks Runtime Version:** Databricks Runtime 5.0 or above
  * **Python Version:** Python 3
1. Install required libraries or if using Databricks Runtime 5.1 or above (but not Databricks Runtime for ML), run Cmd 6.
   1. Create required libraries.
      * Source **PyPI** and enter `mlflow[extras]`. This installs `mlflow` and all its dependencies.
      * Source **PyPI** and enter `azureml-sdk[databricks]`.
   1. Install the libraries into the cluster.
1. Attach this notebook to the cluster.

## 1. Create or load an Azure ML Workspace

Before models can be deployed to Azure ML, you must create or obtain an Azure ML Workspace. The `azureml.core.Workspace.create()` function will load a workspace of a specified name or create one if it does not already exist. For more information about creating an Azure ML Workspace, see the [Azure ML Workspace management documentation](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-manage-workspace).

In [6]:
import azureml
from azureml.core import Workspace

subscription_id = "A RENSEIGNER"
workspace_name = "azuremlservice"
resource_group = "azuremlserviceresourcegroup"
workspace_location="West Europe"

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (cryptography 1.5 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('cryptography>=2.5'), {'paramiko'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core:StepRun._from_reused_dto with exception (cryptography 1.5 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('cryptography>=2.5'), {'paramiko'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core:PipelineRun._from_dto with exception (cryptography 1.5 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('cryptography>=2.5'), {'paramiko'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core:StepRun._from_dto with exception (cryptography 1.5 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('cryptography>=2.5'), {'paramiko'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (cryptography 1.5 (/databricks/python3/lib/python3.5/site-packages), Requirement.parse('cryptography>=2.5'), {'paramiko'}).

In [7]:
workspace = Workspace.create(name = workspace_name,
                             location = workspace_location,
                             resource_group = resource_group,
                             subscription_id = subscription_id,
                             exist_ok=True)

## 2. Build an Azure Container Image for model deployment

### 2.1 Use MLflow to build a Container Image for the trained model

Use the `mlflow.azuereml.build_image` function to build an Azure Container Image for the trained MLflow model. This function also registers the MLflow model with a specified Azure ML workspace. The resulting image can be deployed to Azure Container Instances (ACI) or Azure Kubernetes Service (AKS) for real-time serving.

In [10]:
# Récupération de l'ID du modèle depuis MLflow
run_id1 = "becfa29d52b74ef38230777d620398b0"
model_uri = "runs:/" + run_id1 + "/model"

### Génération image

In [12]:
import mlflow.azureml

# Création de l'image - quelques secondes de traitements
model_image, azure_model = mlflow.azureml.build_image(model_uri=model_uri, 
                                                      workspace=workspace,
                                                      model_name="modelazuremldb",
                                                      image_name="modelazuremldb",
                                                      description="Sklearn ElasticNet image for predicting diabetes progression",
                                                      synchronous=False)

Registering model modelazuremldb
Creating image

In [13]:
# Prévoir 6 minutes de temps de traitement
model_image.wait_for_creation(show_output=True)

Running..................................................................................................
Succeeded
Image creation operation finished for image modelazuremldb:5, operation "Succeeded"

## 3. Deploy the model to "dev" using Azure Container Instances (ACI)

> Plus d'information sur ACI : https://azure.microsoft.com/en-us/services/container-instances/

### 3.1 Create an ACI webservice deployment using the model's Container Image

Using the Azure ML SDK, deploy the Container Image for the trained MLflow model to ACI.

In [17]:
from azureml.core.webservice import AciWebservice, Webservice

dev_webservice_name = "diabetesmodelazuremldb"
dev_webservice_deployment_config = AciWebservice.deploy_configuration()
dev_webservice = Webservice.deploy_from_image(name=dev_webservice_name, image=model_image, deployment_config=dev_webservice_deployment_config, workspace=workspace)

Creating service

Déploiement ACI - prévoir 4 minutes de temps de traitement :

In [19]:
dev_webservice.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"

### 3.2 Visualisation de vos instances ACI

> Pour visualiser vos instances ACI : https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.ContainerInstance%2FcontainerGroups

Exemple :
<img src="https://raw.githubusercontent.com/retkowsky/images/master/ACI2.jpg">

## 4. Query the deployed model in "dev"

### 4.1 Load diabetes dataset

In [25]:
from sklearn import datasets
diabetes = datasets.load_diabetes()

### 4.2 Create sample input vector

In [27]:
import pandas as pd
import numpy as np

X = diabetes.data
y = diabetes.target
Y = np.array([y]).transpose()
d = np.concatenate((X, Y), axis=1)
cols = ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6', 'progression']
data = pd.DataFrame(d, columns=cols)
sample = data.drop(["progression"], axis=1).iloc[[0]]
                                                 
query_input = sample.to_json(orient='split')
query_input = eval(query_input)
query_input.pop('index', None)

Out[ 9 ]: [0]

### 4.3 Evaluate the sample input vector by sending an HTTP request
Query the ACI webservice's scoring endpoint by sending an HTTP POST request that contains the input vector.

In [29]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

Scoring URL :

In [31]:
dev_webservice.scoring_uri

Out[ 11 ]: 'http://04f91ca2-4c92-42b6-b013-0464a663f2fd.westeurope.azurecontainer.io/score'

Appel du modèle :

In [33]:
dev_prediction = query_endpoint_example(scoring_uri=dev_webservice.scoring_uri, inputs=query_input)

Sending batch prediction request with inputs: {'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]], 'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']}
Received response: [199.95454018245817]

## 5. Deploy the model to production using [Azure Kubernetes Service (AKS)](https://azure.microsoft.com/en-us/services/kubernetes-service/). <br> Option 1 or Option 2.

### Option 1: Create a new AKS cluster

If you do not have an active AKS cluster for model deployment, create one using the Azure ML SDK.

> ##### Prévoir 7 minutes de temps de traitement

In [36]:
from azureml.core.compute import AksCompute, ComputeTarget

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_cluster_name = "diabetes-cluster" 
# Create the cluster
aks_target = ComputeTarget.create(workspace = workspace, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

Creating.....................................................................................................................................................................................................................................................................................................................................
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
None

> Pour visualiser vos clusters AKS : https://ms.portal.azure.com/#blade/HubsExtension/BrowseResourceBlade/resourceType/Microsoft.ContainerService%2FmanagedClusters

Exemple :
<br>
<br>
<img src="https://raw.githubusercontent.com/retkowsky/images/master/AKS.jpg">

### Option 2: Connect to an existing AKS cluster

If you already have an active AKS cluster running, you can add it to your Workspace using the Azure ML SDK.

In [40]:
#from azureml.core.compute import AksCompute, ComputeTarget

# Get the resource group from https://porta..azure.com -> Find your resource group
#resource_group = "<resource-group>"

# Give the cluster a local name
#aks_cluster_name = "diabetes-cluster"

# Attatch the cluster to your workgroup
#attach_config = AksCompute.attach_configuration(resource_group=resource_group, cluster_name=aks_cluster_name)
#aks_target = ComputeTarget.attach(workspace, name="diabetes-compute", attach_config)

# Wait for the operation to complete
#aks_target.wait_for_completion(True)
#print(aks_target.provisioning_state)
#print(aks_target.provisioning_errors)

### 6. Deploy to the model's image to the specified AKS cluster

In [42]:
from azureml.core.webservice import Webservice, AksWebservice

prod_webservice_name = "diabetes-model-prod"
prod_webservice_deployment_config = AksWebservice.deploy_configuration()

prod_webservice = Webservice.deploy_from_image(workspace = workspace, 
                                               name = prod_webservice_name,
                                               image = model_image,
                                               deployment_config = prod_webservice_deployment_config,
                                               deployment_target = aks_target)

Creating service

#### Déploiement

In [44]:
# Prévoir 3 minutes de temps de traitement
prod_webservice.wait_for_deployment(show_output = True)

Running..............
SucceededAKS service creation operation finished, operation "Succeeded"

## 7. Query the deployed model in production

#### Evaluate the sample input vector by sending an HTTP request
Query the AKS webservice's scoring endpoint by sending an HTTP POST request that includes the input vector. The production AKS deployment may require an authorization token (service key) for queries. Include this key in the HTTP request header.

In [47]:
import requests
import json

def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=json.dumps(inputs), headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

#### Scoring URL :

In [49]:
prod_scoring_uri = prod_webservice.scoring_uri
prod_service_key = prod_webservice.get_keys()[0] if len(prod_webservice.get_keys()) > 0 else None

### Test URL :

In [51]:
prod_prediction1 = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=query_input)

Sending batch prediction request with inputs: {'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]], 'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']}
Received response: [199.95454018245817]

## 8. Update the production deployment

### Build an Azure Container Image for the new model

In [54]:
run_id2 = "ae67ad7ed7bf4ce09900333db390780c"
model_uri = "runs:/" + run_id2 + "/model"

In [55]:
import mlflow.azureml

model_image_updated, azure_model_updated = mlflow.azureml.build_image(model_uri=model_uri, 
                                                                      workspace=workspace,
                                                                      model_name="model-updated",
                                                                      image_name="model-updated",
                                                                      description="Sklearn ElasticNet image for predicting diabetes progression",
                                                                      synchronous=False)

Registering model model-updated
Creating image

In [56]:
# Prévoir 6 minutes de temps de traitement
model_image_updated.wait_for_creation(show_output=True)

NotStarted......................................................................
Succeeded
Image creation operation finished for image model-updated:3, operation "Succeeded"

Visualisation : https://ms.portal.azure.com/#blade/HubsExtension/Resources/resourceType/Microsoft.ContainerInstance%2FcontainerGroups

### Deploy the new model's image to the AKS cluster

Using the [`azureml.core.webservice.AksWebservice.update()`](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice.akswebservice?view=azure-ml-py#update) function, replace the deployment's existing model image with the new model image.

In [59]:
prod_webservice.update(image=model_image_updated)

In [60]:
# 2 minutes de temps de traitement
prod_webservice.wait_for_deployment(show_output = True)

Running.................
SucceededAKS service creation operation finished, operation "Succeeded"

### Query the updated model

In [62]:
prod_prediction2 = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=query_input)

Sending batch prediction request with inputs: {'data': [[0.0380759064, 0.0506801187, 0.0616962065, 0.021872355, -0.0442234984, -0.0348207628, -0.0434008457, -0.002592262, 0.0199084209, -0.0176461252]], 'columns': ['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']}
Received response: [180.2072757797947]

## 9. Compare the predictions

Simulation 1

In [65]:
print("Simulation 1 :")
print("Run ID: {} - Prediction = {}".format(run_id1, prod_prediction1)) 

Simulation 1 :
Run ID: becfa29d52b74ef38230777d620398b0 - Prediction = [199.95454018245817]

Simulation 2

In [67]:
print("Simulation 2 :")
print("Run ID: {} - Prediction = {}".format(run_id2, prod_prediction2))

Simulation 2 :
Run ID: ae67ad7ed7bf4ce09900333db390780c - Prediction = [180.2072757797947]

## 10. Clean up the deployments

### Terminate the "dev" ACI webservice

Because ACI manages compute resources on your behalf, deleting the "dev" ACI webservice will remove all resources associated with the "dev" model deployment

In [70]:
dev_webservice.delete()

### Terminate the production AKS webservice

This terminates the real-time serving webservice running on the specified AKS cluster. It **does not** terminate the AKS cluster.

In [72]:
prod_webservice.delete()

### Remove the AKS cluster from the Azure ML Workspace

If the cluster was created using the Azure ML SDK (see **Option 1: Create a new AKS cluster**), remove it from the Azure ML Workspace will terminate the cluster, including all of its compute resources and deployments.

If the cluster was created independently (see **Option 2: Connect to an existing AKS cluster**), it will remain active after removal from the Azure ML Workspace.

In [74]:
aks_target.delete()

> Fin